# Example 1: Reading Netezza data using JDBC/Panda dataframe

In [ ]:
# @hidden_cell
# This connection object is used to access your data and contains your credentials.
# You might want to remove those credentials before you share your notebook.

# This example uses the data connection configured in the project

from project_lib import Project
project = Project.access()
weather_credentials = project.get_connected_data(name="weather")

import jaydebeapi, pandas as pd
weather_connection = jaydebeapi.connect('org.netezza.Driver',
    '{}://{}:{}/{}'.format('jdbc:netezza',
    weather_credentials['host'],
    weather_credentials['port'],
    weather_credentials['database']),
    [weather_credentials['username'],
    weather_credentials['password']])

query = 'SELECT * FROM ADMIN.WEATHER'
data_df_1 = pd.read_sql(query, con=weather_connection)
data_df_1.head()

# You can close the database connection with the following code.
# weather_connection.close()
# To learn more about the jaydebeapi package, please read the documentation: https://pypi.org/project/JayDeBeApi/


# Example 2: Using Netezza ODBC driver to access Netezza database

In [ ]:
# Install in internet connected cluster
# !pip install nzalchemy

# Install procedure for airgapped cluster using locally downloaded package
!pip install nzalchemy --no-index --find-links file:/user-home/_global_/python-3/nzalchemy
    
from sqlalchemy import create_engine, MetaData, Table, Column, select
import nzalchemy as nz


In [ ]:
# Note: This libnzodbc.so file was copied into nginx pod under the path /user-home/_global_/dbdrivers/netezza/lib64
# 

import urllib 
params= urllib.parse.quote_plus("DRIVER=/user-home/_global_/dbdrivers/netezza/lib64/libnzodbc.so;SERVER=9.46.76.244;PORT=5480;DATABASE=WEATHER;UID=admin;PWD=password")

# engine = create_engine("netezza+pyodbc:///?odbc_connect=%s" % params,  echo=True)
print(params)

In [ ]:
engine = create_engine("netezza+pyodbc:///?odbc_connect=%s" % params,  echo=True)

#### Sample code 
meta = MetaData()
test = Table(
'TEST', meta,
Column('id', nz.INTEGER),
Column('name', nz.VARCHAR(20) ),
Column('gender', nz.CHAR),
)
meta.create_all(engine)
#conn for insert and select
conn = engine.connect()
#Insert 
conn.execute(test.insert(),[
			{'id':2,'name':'xyz','gender':'F'},
			{'id':3,'name':'abc','gender':'M'},
			]
		)

meta.create_all(engine)

In [ ]:
conn = engine.connect()

In [ ]:
# Qeuery  data from the sample database
result = conn.execute("select * from weather")

In [ ]:
for row in result:
        print (row)

# Example 3: NZ Alchemy in reading and writing into Netezza DB

In [ ]:
# Reading Netezza DB using ODBC driver into panda dataframe

import pandas as pd
import pyodbc

df = pd.read_sql("select * from weather", con=engine)
df.head(5)
                          

In [ ]:
from sqlalchemy.types import BIGINT
from sqlalchemy.types import BOOLEAN
from sqlalchemy.types import CHAR
from sqlalchemy.types import DATE
from sqlalchemy.types import FLOAT
from sqlalchemy.types import INTEGER
from sqlalchemy.types import NUMERIC
from sqlalchemy.types import REAL
from sqlalchemy.types import SMALLINT
from sqlalchemy.types import TEXT
from sqlalchemy.types import VARCHAR

In [ ]:
# df.to_sql("weather2", con=engine)
# Note dtype mapping is required for unsupported data type

df.to_sql("weather2", con=engine, chunksize=10000, method='multi', if_exists='append', index=False, 
dtype= {'mintemp' :nz.FLOAT(8), 
		'maxtemp' :nz.FLOAT(8), 
	'rainfall' :nz.FLOAT(8), 
	'evaporation' :nz.FLOAT(8), 
	'sunshine' :nz.FLOAT(8), 
	'windgustspeed' :nz.FLOAT(8), 
	'windspeed9am' :nz.FLOAT(8), 
	'windspeed3pm' :nz.FLOAT(8), 
	'humidity9am' :nz.FLOAT(8), 
	'humidity3pm' :nz.FLOAT(8), 
	'pressure9am' :nz.FLOAT(8), 
	'pressure3pm' :nz.FLOAT(8), 
	'temp9am' :nz.FLOAT(8), 
	'temp3pm' :nz.FLOAT(8), 
	'risk_mm' :nz.FLOAT(8)
})

# How to Create NZ Table using Panda dataframe and NZAlchemy

In [ ]:
# Read the csv into data frame
data_patient = pd.read_csv('/project_data/data_asset/patientdataV6.csv')
data_patient.head()

In [ ]:
# Write into NZ database
data_patient.to_sql("patient", con=engine)

In [ ]:
# Query the netezza database table
engine.execute("select * from patient").fetchall()

In [ ]:
# For more details refer the nzalchemy documentation https://github.com/IBM/nzalchemy